In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import random 
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path

In [3]:
articles = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/ML Assignment/Processed_Articles.csv")
articles.head() 

,Unnamed: 0,Article,Date,Heading,NewsType,Is_Deadline,Start,End
0,0,karachi: the sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business,0,0,0
1,1,hong kong: asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business,0,0,0
2,2,hong kong: hong kong shares opened 0.66 percen...,1/5/2015,hong kong stocks open 0.66 percent lower,business,1,156,166
3,3,hong kong: asian markets tumbled tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business,1,363,377
4,4,new york: us oil prices monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business,1,1037,1050


In [4]:
final_result = articles[:][articles['Is_Deadline']==1]
for i in range(len(final_result)):
    print(final_result.iloc[i]["Article"][final_result.iloc[i]["Start"]:final_result.iloc[i]["End"]])

03/22/2022
26 march, 2022
13 june, 2022
2022/05/10
04/16/2022
20/03/2022
2022 jun-07 14:44
friday, april 08, 2022 02:44 pm
monday, april 04, 2022 02:44 pm
14 may, 2022
may-16-2022
2022/03/07
2022/05/15
2022/05/30
friday 01. july 2022
sunday, may 29, 2022 02:44 pm
2022-03-05 14:44:02
friday 04. march 2022
20/06/22
2022/02/28
2022-07-26 14:44:02
saturday 04. june 2022
2022/02/26
2022 apr-07 14:44
tuesday 12. july 2022
may-24-2022
27/03/22
jun-16-2022
2022-01-31 14:44:02
2022/03/06
friday, february 04, 2022 02:44 pm
wednesday 19. january 2022
06/18/2022
jul-28-2022
2022-07-08 14:44:02
2022 may-24 14:44
2022-03-26 14:44:02
wednesday 27. april 2022
2022/03/11
2022/04/22
31 july, 2022
2022-04-17 14:44:02
mar-18-2022
13 march, 2022
jul-29-2022
2022 jan-26 14:44
apr-20-2022
09/02/2022
monday 28. february 2022
11/02/22
04 may, 2022
2022/02/14
2022 jul-02 14:44
08 may, 2022
saturday 29. january 2022
27/04/22
08/07/2022
07/06/2022
saturday, july 02, 2022 02:44 pm
thursday 14. july 2022
03 april, 

In [5]:
DATA = []
LABEL = "DATE"
# Training examples in the required format
for index, row in articles.iterrows():
    _, article, _, _, _, is_deadline, start, end = row
    entities = [] if not is_deadline else [(start, end, LABEL)]
    DATA.append((article, {"entities": entities}))
DATA

[('karachi: the sindh government has decided to bring down public transport fares by 7 per cent due to massive reduction in petroleum product prices by the federal government, geo news reported.sources said reduction in fares will be applicable on public transport, rickshaw, taxi and other means of traveling.meanwhile, karachi transport ittehad (kti) has refused to abide by the government decision.kti president irshad bukhari said the commuters are charged the lowest fares in karachi as compare to other parts of the country, adding that 80pc vehicles run on compressed natural gas (cng). bukhari said karachi transporters will cut fares when decrease in cng prices will be made.                        \r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n',
  {'entities': []}),
 ('hong kong: asian markets started 2015 on an upswing in limited trading on friday, with mainland chinese stocks surging in hong kong on speculation beijing may ease monetary policy to boost slowing growth.hong kong rose 1.

In [6]:
def test_train_split(DATA, split=0.8):
    random.shuffle(DATA)
    split_point = int(split*len(DATA))
    TRAIN_DATA = DATA[:split_point]
    TEST_DATA = DATA[split_point:]
    return TRAIN_DATA, TEST_DATA
TRAIN_DATA, TEST_DATA = test_train_split(DATA, split=0.8)

In [7]:
nlp=spacy.load("en_core_web_sm") 
ner=nlp.get_pipe('ner')
nlp

In [8]:
# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
#optimizer = nlp.entity.create_optimizer()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [9]:
# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # Training for 30 iterations     
  for itn in range(10):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # ictionary to store losses
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      # Calling update() over the iteration
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

Streaming output truncated to the last 5000 lines.
Losses {'ner': 1009.1311857306025}
Losses {'ner': 1009.1311857325886}
Losses {'ner': 1009.1350251043682}
Losses {'ner': 1009.135027601586}
Losses {'ner': 1011.1792555711879}
Losses {'ner': 1011.179766460198}
Losses {'ner': 1011.1797664719109}
Losses {'ner': 1011.1798030539234}
Losses {'ner': 1011.2797278269034}
Losses {'ner': 1011.279729315561}
Losses {'ner': 1011.3093439873414}
Losses {'ner': 1017.2110190512567}
Losses {'ner': 1017.2114118958356}
Losses {'ner': 1017.2114119028806}
Losses {'ner': 1017.2114126009112}
Losses {'ner': 1017.2114134455161}
Losses {'ner': 1017.2114134961096}
Losses {'ner': 1017.2495386820258}
Losses {'ner': 1017.2495386823351}
Losses {'ner': 1017.2495386954449}
Losses {'ner': 1017.5810917731566}
Losses {'ner': 1017.5811002523693}
Losses {'ner': 1017.5811002523811}
Losses {'ner': 1017.5811002812933}
Losses {'ner': 1017.581100308977}
Losses {'ner': 1017.5811004477499}
Losses {'ner': 1017.5811004499436}
Losses {

In [10]:
output_dir = Path("/content/gdrive/MyDrive/Colab Notebooks/ML Assignment/")
nlp.meta['name'] = "Date_Recognizer"
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content/gdrive/MyDrive/Colab Notebooks/ML Assignment


In [11]:
# Test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
#doc2 = nlp2()
#for ent in doc2.ents:
#    print(ent.label_, ent.text)

Loading from /content/gdrive/MyDrive/Colab Notebooks/ML Assignment


In [12]:
# catastrophic forgetting for NLP

In [36]:
final_test = [ t for t in TEST_DATA if not t[1]['entities'] ]
for d in final_test:
    t, e = d
    doc2 = nlp2(t)
    for ent in doc2.ents:
        print(f"{ent.text:10} {e['entities']}")

top-10     []


In [31]:
final_test = [ t for t in TEST_DATA if t[1]['entities'] ]
for d in final_test:
    t, e = d
    doc2 = nlp2(t)
    for ent in doc2.ents:
        print(f"{ent.text:40} {t[e['entities'][0][0]:e['entities'][0][1]]}")

20/04/22                                 20/04/22
23/07/22                                 23/07/22
06/08/22                                 06/08/22
06/03/2022                               06/03/2022
jul-15-2022                              jul-15-2022
2022/03/22                               2022/03/22
friday 08.                               friday 08. april 2022
2022 jan-23 14:44                        2022 jan-23 14:44
05/13/2022                               05/13/2022
mar-01-2022                              mar-01-2022
2022-04-09 14:44:02                      2022-04-09 14:44:02
06/23/2022                               06/23/2022
03/07/2022                               03/07/2022
2022 mar-03 14:44                        2022 mar-03 14:44
06 june, 2022                            06 june, 2022
02/14/2022                               02/14/2022
17/06/2022                               17/06/2022
2022/03/27                               2022/03/27
09/06/22                       